# Can I plot the single-cell tracks and glimpses

Using the outliers that are 1sd away from the mean of dMtb

In [1]:
import os, glob
import btrack
import json
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from macrohet import dataio, tools, visualise
from tqdm.auto import tqdm
from scipy import stats
import pickle
import re
import cv2
from skimage import io
colors = sns.set_palette('PiYG')
track_scale_factor = 5.04

### Load metadata

In [2]:
base_dir = '/mnt/DATA/macrohet/'
# base_dir = '/Volumes/lab-gutierrezm/home/users/dayn/macrohet/'
metadata_fn = os.path.join(base_dir, 'macrohet_images/Index.idx.xml')
metadata = dataio.read_harmony_metadata(metadata_fn)  
metadata

Reading metadata XML file...


Extracting HarmonyV5 metadata:   0%|          | 0/113400 [00:00<?, ?it/s]

Extracting metadata complete!


,id,State,URL,Row,Col,FieldID,PlaneID,TimepointID,ChannelID,FlimID,...,PositionZ,AbsPositionZ,MeasurementTimeOffset,AbsTime,MainExcitationWavelength,MainEmissionWavelength,ObjectiveMagnification,ObjectiveNA,ExposureTime,OrientationMatrix
0,0303K1F1P1R1,Ok,r03c03f01p01-ch1sk1fk1fl1.tiff,3,3,1,1,0,1,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
1,0303K1F1P1R2,Ok,r03c03f01p01-ch2sk1fk1fl1.tiff,3,3,1,1,0,2,1,...,0,0.135583505,0,2021-04-16T19:09:33.84+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
2,0303K1F1P2R1,Ok,r03c03f01p02-ch1sk1fk1fl1.tiff,3,3,1,2,0,1,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
3,0303K1F1P2R2,Ok,r03c03f01p02-ch2sk1fk1fl1.tiff,3,3,1,2,0,2,1,...,2E-06,0.135585502,0,2021-04-16T19:09:34.12+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
4,0303K1F1P3R1,Ok,r03c03f01p03-ch1sk1fk1fl1.tiff,3,3,1,3,0,1,1,...,4E-06,0.135587499,0,2021-04-16T19:09:34.4+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113395,0609K75F9P1R2,Ok,r06c09f09p01-ch2sk75fk1fl1.tiff,6,9,9,1,74,2,1,...,0,0.135533601,266399.61,2021-04-19T21:14:19.477+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113396,0609K75F9P2R1,Ok,r06c09f09p02-ch1sk75fk1fl1.tiff,6,9,9,2,74,1,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113397,0609K75F9P2R2,Ok,r06c09f09p02-ch2sk75fk1fl1.tiff,6,9,9,2,74,2,1,...,2E-06,0.135535598,266399.61,2021-04-19T21:14:19.757+01:00,640,706,40,1.1,0.2,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."
113398,0609K75F9P3R1,Ok,r06c09f09p03-ch1sk75fk1fl1.tiff,6,9,9,3,74,1,1,...,4E-06,0.135537595,266399.61,2021-04-19T21:14:20.037+01:00,488,522,40,1.1,0.1,"[[0.990860,0,0,-15.9],[0,-0.990860,0,-44.8],[0..."


In [3]:
base_dir = '/mnt/DATA/macrohet/'
metadata_path = os.path.join(base_dir, 'macrohet_images/Assaylayout/20210602_Live_cell_IPSDMGFP_ATB.xml')
assay_layout = dataio.read_harmony_metadata(metadata_path, assay_layout=True,)
assay_layout

Reading metadata XML file...
Extracting metadata complete!


Strain Compound Concentration ConcentrationEC
3 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
4 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA            60            EC50
  7     WT      RIF           0.1            EC50
  8     WT      INH          0.04            EC50
  9     WT      BDQ          0.02            EC50
5 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99
6 4    RD1     CTRL             0             EC0
  5     WT     CTRL             0             EC0
  6     WT      PZA           400            EC99
  7     WT      RIF             2            EC99
  8     WT      INH             2            EC99
  9     WT      BDQ           2.5            EC99

### Load cells from outlier dMtb analysis

In [4]:
# load glimpses from outliers and extract unique_IDs from them
outlier_fns = glob.glob('/mnt/DATA/macrohet/results/glimpses/interactive_plots/dMtb/*.png')
unique_IDs = [fn.split('_')[2] for fn in outlier_fns]

In [5]:
unique_IDs

['701.6.5',
 '421.3.9',
 '416.5.7',
 '166.4.6',
 '2198.5.9',
 '184.4.5',
 '701.3.4',
 '1138.3.5',
 '1809.3.9',
 '1254.4.5',
 '82.4.7',
 '1578.5.9',
 '609.4.4',
 '535.3.7',
 '114.6.7',
 '91.5.7',
 '70.5.5',
 '510.4.5',
 '2201.3.9',
 '2135.3.4',
 '2144.3.8',
 '78.4.6',
 '1894.6.5',
 '279.6.8',
 '331.3.8',
 '640.4.4',
 '513.6.8',
 '354.5.9',
 '504.5.5',
 '1254.3.6',
 '223.3.5',
 '467.5.5',
 '1118.4.7',
 '904.4.5',
 '320.4.4',
 '839.3.8',
 '1322.5.5',
 '2088.3.7',
 '546.6.4',
 '749.5.7',
 '461.4.8',
 '1805.4.6',
 '182.4.4',
 '1693.5.7',
 '905.3.8',
 '2190.6.6',
 '24.6.9',
 '2135.3.4',
 '1418.6.4',
 '1254.4.5',
 '1472.4.4',
 '78.4.6',
 '2202.3.8',
 '167.4.4',
 '314.4.7',
 '621.4.5',
 '546.6.4',
 '2180.6.5',
 '1404.5.6',
 '150.4.5',
 '897.6.8',
 '659.3.4',
 '705.5.7',
 '360.3.7',
 '902.4.9',
 '362.3.5',
 '1075.6.5',
 '470.5.8',
 '304.4.4',
 '353.6.5',
 '1065.5.4',
 '676.3.9',
 '2549.3.8',
 '688.5.7',
 '235.5.7',
 '143.4.7',
 '1418.6.4',
 '493.6.8',
 '1035.3.9',
 '299.3.6',
 '128.3.5',
 '905.

In [6]:
len(unique_IDs)

664

# Load sc_df

In [7]:
# Specify the path to your pickle file
sc_df_path = "/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_>40_interp.pkl" 
#"/mnt/DATA/macrohet/results/preliminary_sc_measures/sc_dfs/sc_df_>40.pkl"

# Load the pickle file
with open(sc_df_path, "rb") as file:
    df = pickle.load(file)

In [8]:
# subset_df = df[df['Unique_ID'].isin(true_unique_IDs)] # only GT 3.5 tracks
subset_df = df[df['Unique ID'].isin(unique_IDs)] # all outlier tracks for now

In [9]:
subset_df

,Time (hours),x,y,x scaled,y scaled,Infection status,Initial infection status,Final infection status,Area,Intracellular mean Mtb content,...,Eccentricity,MSD,Strain,Compound,Concentration,Cell ID,Acquisition ID,Unique ID,Technical replicate,dMtb
0,0,708.125061,77.589088,3568.950308,391.049006,0.0,0.0,1.0,42556.0,365.408905,...,0.813310,0.000000,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
1,1,725.449097,73.245003,3656.263447,369.154814,0.0,0.0,1.0,48727.0,362.852478,...,0.896474,17.860383,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
2,2,723.088928,76.431496,3644.368198,385.214738,0.0,0.0,1.0,43484.0,363.515442,...,0.876911,3.965367,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
3,3,726.009766,79.395073,3659.089219,400.151168,0.0,0.0,1.0,39113.0,360.448578,...,0.717504,4.161019,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
4,4,728.209656,82.197769,3670.176665,414.276757,0.0,0.0,1.0,32644.0,362.289764,...,0.425791,3.562951,RD1,CTRL,EC0,433,"(3, 4)",433.3.4,1.0,0.095232
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
697433,70,565.650940,1081.278931,2850.880737,5449.645811,0.0,1.0,0.0,61587.0,374.370483,...,0.521127,0.963656,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031
697434,71,566.611755,1083.470947,2855.723247,5460.693574,0.0,1.0,0.0,58778.0,372.559570,...,0.519641,2.393346,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031
697435,72,568.148987,1081.411011,2863.470894,5450.311494,0.0,1.0,0.0,64893.0,372.814087,...,0.579433,2.570296,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031
697436,73,570.763306,1084.552124,2876.647061,5466.142705,1.0,1.0,0.0,61917.0,374.565948,...,0.554528,4.086717,WT,BDQ,EC99,1202,"(6, 9)",1202.6.9,2.0,-0.835031


In [10]:
from macrohet import tile, dataio, visualise, aws_tools
from skimage.io import imsave
import os

In [11]:
global previous_row, previous_column
previous_row, previous_column = 0,0

In [16]:
import multiprocessing

# Define your cropping and saving function
def crop_and_save(unique_ID):
    global previous_row, previous_column
    cell_ID, row, column = unique_ID.split('.') 
    time = subset_df[subset_df['Unique ID'] == unique_ID]['Time (hours)'].values
    if (row, column) != (previous_row, previous_column):
        # load segmentation
#         print(f'Loading new set of segmentation for position {row}, {column}')
        with btrack.io.HDF5FileHandler(os.path.join(base_dir,
                                                    f'labels/macrohet_seg_model/{int(row),int(column)}.h5'),
                                       'r',
                                       obj_type='obj_type_1') as reader:
            segmentation = reader.segmentation
    for t in tqdm(time, desc = f'Iterating over {len(time)} time points in cell ID {unique_ID}', leave = False):
        t = int(t)
        output_fn = f'/mnt/DATA/macrohet/results/glimpses/interactive_plots/time_series/glimpse_{cell_ID}.{row}.{column}_t{t}.png'
        if os.path.exists(output_fn):
#             print(f'{unique_ID} glimpse already created at time {t}, continuing to next time point')
            continue
        glimpse = visualise.tn_glimpse_maker(unique_ID, 
                                             df, 
                                             time=t, 
                                             metadata=metadata, 
                                             segmentation=segmentation)
        imsave(output_fn, glimpse)
        basename = os.path.basename(output_fn)
        aws_fn = f'time_series/all_glimpses/{basename}'
        try:
            aws_tools.upload(output_fn, 
                             'macrohet', 
                                aws_fn, output_msg = False)
        except:
            non_upload_list.append(basename)
    previous_row, previous_column = row, column

# Define the number of processes to run in parallel
num_processes = multiprocessing.cpu_count()  - 2 # Use the number of available CPU cores

# Create a Pool of worker processes
pool = multiprocessing.Pool(processes=num_processes)


[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 7).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 5).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 4).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 6).h5...
[INFO][2023/06/27 02:30:23 PM] Opening HDF file: /mnt/DATA/macrohet/labels/macrohet_seg_model/(3, 8).h5...
[INFO][2023/06/27 02:30:23 PM] Openin

In [ ]:

# Apply the crop_and_save function to each image in parallel
results = pool.map(crop_and_save, subset_df['Unique ID'].unique())

# Close the pool to prevent any more tasks from being submitted
pool.close()

# Wait for all processes to complete
pool.join()

# Process the results if needed